In [1]:
import numpy as np
import pandas as pd
import pyspark.sql.functions as f
import pyspark.sql.types as t
from gentropy.common.session import Session
from gentropy.dataset.ld_index import LDIndex
from gentropy.dataset.study_index import StudyIndex
from gentropy.dataset.study_locus import StudyLocus
from gentropy.dataset.summary_statistics import SummaryStatistics
from gentropy.locus_breaker_clumping import LocusBreakerClumpingStep
from gentropy.method.locus_breaker_clumping import LocusBreakerClumping
from gentropy.method.susie_inf import SUSIE_inf
from gentropy.method.window_based_clumping import WindowBasedClumping
from pyspark.sql.window import Window


Loading BokehJS ...

In [2]:
session = Session(start_hail=True,
                  hail_home="/Users/dc16/Library/Caches/pypoetry/virtualenvs/gentropy-iQynFIia-py3.10/lib/python3.10/site-packages/hail",
                  extended_spark_conf={"spark.driver.memory": "10g",
                                       "spark.driver.maxResultSize": "10g",})


24/08/07 11:39:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/08/07 11:39:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/backend/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/backend/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 3.3.4
SparkUI available at http://mib117960s.internal.sanger.ac.uk:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.127-bb535cd096c5
LOGGING: writing to /dev/null


In [3]:
gwas_cat_cs = StudyLocus.from_parquet(session, "/Users/dc16/output/gwas_cat_fm")
gwas_cat_cs.df = gwas_cat_cs.df.filter(~f.isnan("credibleSetlog10BF"))
ukb_ppp_cs = StudyLocus.from_parquet(session, "/Users/dc16/output/ukb_ppp_fm/")
ukb_ppp_cs.df = ukb_ppp_cs.df.filter(~f.isnan("credibleSetlog10BF"))

si = StudyIndex.from_parquet(session, "/Users/dc16/data/study_index/", recursiveFileLookup=True)
ld = LDIndex.from_parquet(session, "/Users/dc16/data/ld_index/")

In [6]:
gwas_cat_result = SUSIE_inf.credible_set_qc(gwas_cat_cs, si, ld, 1e-5, 0.25, 0.8)
ukb_ppp_result = SUSIE_inf.credible_set_qc(ukb_ppp_cs, si, ld, 1.7e-12, 0.25, 0.8)

In [7]:
gwas_cat_result.df.write.parquet("/Users/dc16/output/gwas_cat_fm_clean", mode="overwrite")
ukb_ppp_result.df.write.parquet("/Users/dc16/output/ukb_ppp_fm_clean", mode="overwrite")

In [21]:
( 
    session.spark.read.parquet(
        "/Users/dc16/output/gwas_cat_fm_clean"
    )
    .distinct()
    .count()
)

220670